In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [5]:
# Age的缺失值由平均数填充

mean_val = train['Age'].mean()
train['Age'].fillna(mean_val, inplace = True)

mean_val2 = test['Age'].mean()
test['Age'].fillna(mean_val2, inplace = True)

In [6]:
numeric = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

In [7]:
# 为了定义每列numerical variable，我们将每列column中的每个数都与中位数进行对比，大于中位数为富人（1），其余为穷人（0）

for column in train[numeric]:
    for i in range(len(train[column])):
        if train[column][i] > train[column].median():
            train[column][i] = 1
        else:
            train[column][i] = 0

for column in test[numeric]:
    for i in range(len(test[column])):
        if test[column][i] > test[column].median():
            test[column][i] = 1
        else:
            test[column][i] = 0

<ipython-input-7-15e6fac4ede8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[column][i] = 0
<ipython-input-7-15e6fac4ede8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[column][i] = 1
<ipython-input-7-15e6fac4ede8>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[column][i] = 0
<ipython-input-7-15e6fac4ede8>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the c

In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8693 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8693 non-null   float64
 8   FoodCourt     8693 non-null   float64
 9   ShoppingMall  8693 non-null   float64
 10  Spa           8693 non-null   float64
 11  VRDeck        8693 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4277 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4277 non-null   float64
 8   FoodCourt     4277 non-null   float64
 9   ShoppingMall  4277 non-null   float64
 10  Spa           4277 non-null   float64
 11  VRDeck        4277 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [10]:
train['Cabin'].str.split(pat="/",expand=True)

,0,1,2
0,B,0,P
1,F,0,S
2,A,0,S
3,A,0,S
4,F,1,S
...,...,...,...
8688,A,98,P
8689,G,1499,S
8690,G,1500,S
8691,E,608,S


In [11]:
train['Cabin1'] = train['Cabin'].str.split(pat="/",expand=True)[0]
train['Cabin2'] = train['Cabin'].str.split(pat="/",expand=True)[1]
train['Cabin3'] = train['Cabin'].str.split(pat="/",expand=True)[2]

In [12]:
test['Cabin'].str.split(pat="/",expand=True)

,0,1,2
0,G,3,S
1,F,4,S
2,C,0,S
3,C,1,S
4,F,5,S
...,...,...,...
4272,G,1496,S
4273,NaN,NaN,NaN
4274,D,296,P
4275,D,297,P


In [13]:
test['Cabin1'] = test['Cabin'].str.split(pat="/",expand=True)[0]
test['Cabin2'] = test['Cabin'].str.split(pat="/",expand=True)[1]
test['Cabin3'] = test['Cabin'].str.split(pat="/",expand=True)[2]

In [14]:
from IPython.display import display
data1 = train[['Cabin1', 'Cabin2', 'Cabin3','HomePlanet','Destination']] 
data2 = test[['Cabin1', 'Cabin2', 'Cabin3','HomePlanet','Destination']] 
display(data1.head())
display(data2.head())

,Cabin1,Cabin2,Cabin3,HomePlanet,Destination
0,B,0,P,Europa,TRAPPIST-1e
1,F,0,S,Earth,TRAPPIST-1e
2,A,0,S,Europa,TRAPPIST-1e
3,A,0,S,Europa,TRAPPIST-1e
4,F,1,S,Earth,TRAPPIST-1e


,Cabin1,Cabin2,Cabin3,HomePlanet,Destination
0,G,3,S,Earth,TRAPPIST-1e
1,F,4,S,Earth,TRAPPIST-1e
2,C,0,S,Europa,55 Cancri e
3,C,1,S,Europa,TRAPPIST-1e
4,F,5,S,Earth,TRAPPIST-1e


In [15]:
data_dummies_1 = pd.get_dummies(data1) 
data_dummies_1.head()

,Cabin1_A,Cabin1_B,Cabin1_C,Cabin1_D,Cabin1_E,Cabin1_F,Cabin1_G,Cabin1_T,Cabin2_0,Cabin2_1,...,Cabin2_998,Cabin2_999,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0,1,0,0,0,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,1
1,0,0,0,0,0,1,0,0,1,0,...,0,0,0,1,1,0,0,0,0,1
2,1,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,1
3,1,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,1
4,0,0,0,0,0,1,0,0,0,1,...,0,0,0,1,1,0,0,0,0,1


In [16]:
data_dummies_2 = pd.get_dummies(data2) 
data_dummies_2.head()

,Cabin1_A,Cabin1_B,Cabin1_C,Cabin1_D,Cabin1_E,Cabin1_F,Cabin1_G,Cabin1_T,Cabin2_0,Cabin2_1,...,Cabin2_996,Cabin2_997,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,0,0,1
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
2,0,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,1,0,0
3,0,0,1,0,0,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,1
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1


# Update

In [17]:
train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Cabin1,Cabin2,Cabin3
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,B,0,P
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,1.0,1.0,1.0,1.0,1.0,Juanna Vines,True,F,0,S
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,1.0,1.0,0.0,1.0,1.0,Altark Susent,False,A,0,S
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1.0,1.0,1.0,1.0,Solam Susent,False,A,0,S
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,1.0,1.0,1.0,1.0,1.0,Willy Santantines,True,F,1,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,1.0,0.0,1.0,1.0,Gravior Noxnuther,False,A,98,P
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,G,1499,S
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1.0,1.0,0.0,Fayey Connon,True,G,1500,S
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1.0,0.0,1.0,1.0,Celeon Hontichre,False,E,608,S


In [18]:
test

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Cabin1,Cabin2,Cabin3
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.000000,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,G,3,S
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.000000,False,0.0,1.0,0.0,1.0,0.0,Lerome Peckers,F,4,S
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.000000,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,C,0,S
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.000000,False,0.0,1.0,0.0,1.0,1.0,Meratz Caltilter,C,1,S
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.000000,False,1.0,0.0,1.0,0.0,0.0,Brence Harperez,F,5,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.000000,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter,G,1496,S
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.000000,False,0.0,1.0,1.0,1.0,1.0,Matty Scheron,NaN,NaN,NaN
4274,9271_01,Mars,True,D/296/P,55 Cancri e,28.658146,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore,D,296,P
4275,9273_01,Europa,False,D/297/P,NaN,28.658146,False,0.0,1.0,0.0,0.0,1.0,Kitakan Conale,D,297,P


In [19]:
data3 = train[['Cabin1', 'Cabin3','HomePlanet','Destination']] 
data4 = test[['Cabin1', 'Cabin3','HomePlanet','Destination']] 

In [20]:
data_dummies_3 = pd.get_dummies(data3) 
data_dummies_3.head()

,Cabin1_A,Cabin1_B,Cabin1_C,Cabin1_D,Cabin1_E,Cabin1_F,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1
1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1
3,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1
4,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1


In [21]:
data_dummies_4 = pd.get_dummies(data3) 
data_dummies_4.head()

,Cabin1_A,Cabin1_B,Cabin1_C,Cabin1_D,Cabin1_E,Cabin1_F,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1
1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1
3,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1
4,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1


In [22]:
train = train.join(data_dummies_3)

In [23]:
test = test.join(data_dummies_4)

In [24]:
remove = ["Cabin","Destination","HomePlanet","Name","Cabin1","Cabin2","Cabin3"]

In [25]:
train.drop(remove, inplace = True, axis = 1)

In [26]:
train

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0001_01,False,39.0,False,0.0,0.0,0.0,0.0,0.0,False,...,0,0,1,0,0,1,0,0,0,1
1,0002_01,False,24.0,False,1.0,1.0,1.0,1.0,1.0,True,...,0,0,0,1,1,0,0,0,0,1
2,0003_01,False,58.0,True,1.0,1.0,0.0,1.0,1.0,False,...,0,0,0,1,0,1,0,0,0,1
3,0003_02,False,33.0,False,0.0,1.0,1.0,1.0,1.0,False,...,0,0,0,1,0,1,0,0,0,1
4,0004_01,False,16.0,False,1.0,1.0,1.0,1.0,1.0,True,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,False,41.0,True,0.0,1.0,0.0,1.0,1.0,False,...,0,0,1,0,0,1,0,1,0,0
8689,9278_01,True,18.0,False,0.0,0.0,0.0,0.0,0.0,False,...,1,0,0,1,1,0,0,0,1,0
8690,9279_01,False,26.0,False,0.0,0.0,1.0,1.0,0.0,True,...,1,0,0,1,1,0,0,0,0,1
8691,9280_01,False,32.0,False,0.0,1.0,0.0,1.0,1.0,False,...,0,0,0,1,0,1,0,1,0,0


In [27]:
vip = train["VIP"]

In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                8693 non-null   object 
 1   CryoSleep                  8476 non-null   object 
 2   Age                        8693 non-null   float64
 3   VIP                        8490 non-null   object 
 4   RoomService                8693 non-null   float64
 5   FoodCourt                  8693 non-null   float64
 6   ShoppingMall               8693 non-null   float64
 7   Spa                        8693 non-null   float64
 8   VRDeck                     8693 non-null   float64
 9   Transported                8693 non-null   bool   
 10  Cabin1_A                   8693 non-null   uint8  
 11  Cabin1_B                   8693 non-null   uint8  
 12  Cabin1_C                   8693 non-null   uint8  
 13  Cabin1_D                   8693 non-null   uint8

In [29]:
test.drop(remove, inplace = True, axis = 1)

In [30]:
test

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0013_01,True,27.000000,False,0.0,0.0,0.0,0.0,0.0,0,...,0,0,1,0,0,1,0,0,0,1
1,0018_01,False,19.000000,False,0.0,1.0,0.0,1.0,0.0,0,...,0,0,0,1,1,0,0,0,0,1
2,0019_01,True,31.000000,False,0.0,0.0,0.0,0.0,0.0,1,...,0,0,0,1,0,1,0,0,0,1
3,0021_01,False,38.000000,False,0.0,1.0,0.0,1.0,1.0,1,...,0,0,0,1,0,1,0,0,0,1
4,0023_01,False,20.000000,False,1.0,0.0,1.0,0.0,0.0,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,True,34.000000,False,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,1,1,0,0,0,0,1
4273,9269_01,False,42.000000,False,0.0,1.0,1.0,1.0,1.0,0,...,0,0,0,1,1,0,0,0,0,1
4274,9271_01,True,28.658146,False,0.0,0.0,0.0,0.0,0.0,0,...,0,0,1,0,0,0,1,0,0,1
4275,9273_01,False,28.658146,False,0.0,1.0,0.0,0.0,1.0,0,...,0,0,1,0,0,1,0,0,1,0


In [31]:
vip2 = test["VIP"]

In [32]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                4277 non-null   object 
 1   CryoSleep                  4184 non-null   object 
 2   Age                        4277 non-null   float64
 3   VIP                        4184 non-null   object 
 4   RoomService                4277 non-null   float64
 5   FoodCourt                  4277 non-null   float64
 6   ShoppingMall               4277 non-null   float64
 7   Spa                        4277 non-null   float64
 8   VRDeck                     4277 non-null   float64
 9   Cabin1_A                   4277 non-null   uint8  
 10  Cabin1_B                   4277 non-null   uint8  
 11  Cabin1_C                   4277 non-null   uint8  
 12  Cabin1_D                   4277 non-null   uint8  
 13  Cabin1_E                   4277 non-null   uint8

# train_c

In [33]:
train_c = train
train_c.drop("VIP",inplace = True, axis = 1)

In [34]:
train_c.head()

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0001_01,False,39.0,0.0,0.0,0.0,0.0,0.0,False,0,...,0,0,1,0,0,1,0,0,0,1
1,0002_01,False,24.0,1.0,1.0,1.0,1.0,1.0,True,0,...,0,0,0,1,1,0,0,0,0,1
2,0003_01,False,58.0,1.0,1.0,0.0,1.0,1.0,False,1,...,0,0,0,1,0,1,0,0,0,1
3,0003_02,False,33.0,0.0,1.0,1.0,1.0,1.0,False,1,...,0,0,0,1,0,1,0,0,0,1
4,0004_01,False,16.0,1.0,1.0,1.0,1.0,1.0,True,0,...,0,0,0,1,1,0,0,0,0,1


In [35]:
train_c_test = train_c[train_c.isnull().T.any()]

In [36]:
train_c_train = train_c.dropna()

In [37]:
train_c_train

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0001_01,False,39.0,0.0,0.0,0.0,0.0,0.0,False,0,...,0,0,1,0,0,1,0,0,0,1
1,0002_01,False,24.0,1.0,1.0,1.0,1.0,1.0,True,0,...,0,0,0,1,1,0,0,0,0,1
2,0003_01,False,58.0,1.0,1.0,0.0,1.0,1.0,False,1,...,0,0,0,1,0,1,0,0,0,1
3,0003_02,False,33.0,0.0,1.0,1.0,1.0,1.0,False,1,...,0,0,0,1,0,1,0,0,0,1
4,0004_01,False,16.0,1.0,1.0,1.0,1.0,1.0,True,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,False,41.0,0.0,1.0,0.0,1.0,1.0,False,1,...,0,0,1,0,0,1,0,1,0,0
8689,9278_01,True,18.0,0.0,0.0,0.0,0.0,0.0,False,0,...,1,0,0,1,1,0,0,0,1,0
8690,9279_01,False,26.0,0.0,0.0,1.0,1.0,0.0,True,0,...,1,0,0,1,1,0,0,0,0,1
8691,9280_01,False,32.0,0.0,1.0,0.0,1.0,1.0,False,0,...,0,0,0,1,0,1,0,1,0,0


In [38]:
train_c_test

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
92,0099_02,NaN,2.0,0.0,0.0,0.0,0.0,0.0,True,0,...,1,0,1,0,1,0,0,0,0,1
98,0105_01,NaN,27.0,0.0,0.0,1.0,1.0,1.0,False,0,...,0,0,1,0,1,0,0,0,0,1
104,0110_02,NaN,40.0,0.0,1.0,0.0,0.0,1.0,False,0,...,0,0,1,0,0,1,0,0,0,1
111,0115_01,NaN,26.0,0.0,0.0,0.0,0.0,0.0,True,0,...,0,0,1,0,0,0,1,0,0,1
152,0173_01,NaN,58.0,0.0,1.0,0.0,1.0,0.0,True,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8620,9197_01,NaN,44.0,0.0,0.0,0.0,0.0,0.0,True,0,...,0,0,1,0,0,1,0,1,0,0
8651,9227_05,NaN,8.0,0.0,0.0,0.0,0.0,0.0,False,0,...,1,0,1,0,1,0,0,0,0,1
8664,9246_01,NaN,32.0,0.0,0.0,0.0,0.0,0.0,True,0,...,1,0,0,1,1,0,0,0,0,1
8675,9259_01,NaN,44.0,1.0,1.0,0.0,1.0,0.0,True,0,...,0,0,1,0,1,0,0,0,0,1


In [39]:
train_c_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 92 to 8687
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                217 non-null    object 
 1   CryoSleep                  0 non-null      object 
 2   Age                        217 non-null    float64
 3   RoomService                217 non-null    float64
 4   FoodCourt                  217 non-null    float64
 5   ShoppingMall               217 non-null    float64
 6   Spa                        217 non-null    float64
 7   VRDeck                     217 non-null    float64
 8   Transported                217 non-null    bool   
 9   Cabin1_A                   217 non-null    uint8  
 10  Cabin1_B                   217 non-null    uint8  
 11  Cabin1_C                   217 non-null    uint8  
 12  Cabin1_D                   217 non-null    uint8  
 13  Cabin1_E                   217 non-null    uint8

In [40]:
y_train = train_c_train["CryoSleep"].astype('int')

In [41]:
y_train

0       0
1       0
2       0
3       0
4       0
       ..
8688    0
8689    1
8690    0
8691    0
8692    0
Name: CryoSleep, Length: 8476, dtype: int32

In [42]:
df_train_c = train_c_train.drop(["CryoSleep","PassengerId"], axis = 1)
df_train_c.astype('int')

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,Cabin1_B,Cabin1_C,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,39,0,0,0,0,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,1
1,24,1,1,1,1,1,1,0,0,0,...,0,0,0,1,1,0,0,0,0,1
2,58,1,1,0,1,1,0,1,0,0,...,0,0,0,1,0,1,0,0,0,1
3,33,0,1,1,1,1,0,1,0,0,...,0,0,0,1,0,1,0,0,0,1
4,16,1,1,1,1,1,1,0,0,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,41,0,1,0,1,1,0,1,0,0,...,0,0,1,0,0,1,0,1,0,0
8689,18,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,1,0
8690,26,0,0,1,1,0,1,0,0,0,...,1,0,0,1,1,0,0,0,0,1
8691,32,0,1,0,1,1,0,0,0,0,...,0,0,0,1,0,1,0,1,0,0


In [43]:
X_train = df_train_c

In [44]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state = 0) 
tree.fit(X_train, y_train) 
print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train))) 

Accuracy on training set: 0.972


In [45]:
df_test_c = train_c_test.drop(["CryoSleep","PassengerId"], axis = 1)
df_test_c.astype('int')

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,Cabin1_B,Cabin1_C,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
92,2,0,0,0,0,0,1,0,0,0,...,1,0,1,0,1,0,0,0,0,1
98,27,0,0,1,1,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
104,40,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,0,1
111,26,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,1
152,58,0,1,0,1,0,1,0,0,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8620,44,0,0,0,0,0,1,0,0,1,...,0,0,1,0,0,1,0,1,0,0
8651,8,0,0,0,0,0,0,0,0,0,...,1,0,1,0,1,0,0,0,0,1
8664,32,0,0,0,0,0,1,0,0,0,...,1,0,0,1,1,0,0,0,0,1
8675,44,1,1,0,1,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,1


In [46]:
X_test = df_test_c

In [47]:
y_pred = tree.predict(X_test) 
print("Test set predictions:\n {}".format(y_pred))

Test set predictions:
 [1 0 0 1 0 1 0 1 0 1 0 0 0 1 0 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 1 1 0 1
 1 1 0 1 1 1 0 0 0 1 1 0 1 0 0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 1
 1 0 0 1 0 1 0 0 0 1 1 1 1 0 1 0 0 0 1 0 1 0 1 1 1 0 0 0 0 0 1 0 1 1 0 1 0
 0 0 0 1 1 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0 0 1 0 0 1 1 0 0 1 1 0 1 1 0 0 0
 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 1 1 0 1 0 0]


In [48]:
y_pred = pd.DataFrame(y_pred)

In [49]:
y_pred.columns = ["CryoSleep"]
y_pred

,CryoSleep
0,1
1,0
2,0
3,1
4,0
...,...
212,1
213,0
214,1
215,0


In [50]:
y_pred_1 = y_pred.set_index(train_c_test.index)

In [51]:
y_pred_1

,CryoSleep
92,1
98,0
104,0
111,1
152,0
...,...
8620,1
8651,0
8664,1
8675,0


In [52]:
train_c_test = train_c_test.fillna(y_pred_1)

In [53]:
train_c_test["CryoSleep"] = train_c_test["CryoSleep"].astype("int")

In [54]:
train_c_test 

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
92,0099_02,1,2.0,0.0,0.0,0.0,0.0,0.0,True,0,...,1,0,1,0,1,0,0,0,0,1
98,0105_01,0,27.0,0.0,0.0,1.0,1.0,1.0,False,0,...,0,0,1,0,1,0,0,0,0,1
104,0110_02,0,40.0,0.0,1.0,0.0,0.0,1.0,False,0,...,0,0,1,0,0,1,0,0,0,1
111,0115_01,1,26.0,0.0,0.0,0.0,0.0,0.0,True,0,...,0,0,1,0,0,0,1,0,0,1
152,0173_01,0,58.0,0.0,1.0,0.0,1.0,0.0,True,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8620,9197_01,1,44.0,0.0,0.0,0.0,0.0,0.0,True,0,...,0,0,1,0,0,1,0,1,0,0
8651,9227_05,0,8.0,0.0,0.0,0.0,0.0,0.0,False,0,...,1,0,1,0,1,0,0,0,0,1
8664,9246_01,1,32.0,0.0,0.0,0.0,0.0,0.0,True,0,...,1,0,0,1,1,0,0,0,0,1
8675,9259_01,0,44.0,1.0,1.0,0.0,1.0,0.0,True,0,...,0,0,1,0,1,0,0,0,0,1


In [55]:
train_c_train["CryoSleep"] = train_c_train["CryoSleep"].astype("int")

<ipython-input-55-d22943eb2e96>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_c_train["CryoSleep"] = train_c_train["CryoSleep"].astype("int")


In [56]:
train_c_train

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0001_01,0,39.0,0.0,0.0,0.0,0.0,0.0,False,0,...,0,0,1,0,0,1,0,0,0,1
1,0002_01,0,24.0,1.0,1.0,1.0,1.0,1.0,True,0,...,0,0,0,1,1,0,0,0,0,1
2,0003_01,0,58.0,1.0,1.0,0.0,1.0,1.0,False,1,...,0,0,0,1,0,1,0,0,0,1
3,0003_02,0,33.0,0.0,1.0,1.0,1.0,1.0,False,1,...,0,0,0,1,0,1,0,0,0,1
4,0004_01,0,16.0,1.0,1.0,1.0,1.0,1.0,True,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0,41.0,0.0,1.0,0.0,1.0,1.0,False,1,...,0,0,1,0,0,1,0,1,0,0
8689,9278_01,1,18.0,0.0,0.0,0.0,0.0,0.0,False,0,...,1,0,0,1,1,0,0,0,1,0
8690,9279_01,0,26.0,0.0,0.0,1.0,1.0,0.0,True,0,...,1,0,0,1,1,0,0,0,0,1
8691,9280_01,0,32.0,0.0,1.0,0.0,1.0,1.0,False,0,...,0,0,0,1,0,1,0,1,0,0


In [57]:
train_c = pd.concat([train_c_train, train_c_test],axis = 0)

In [58]:
train_c = train_c.sort_index()

In [59]:
train_c

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0001_01,0,39.0,0.0,0.0,0.0,0.0,0.0,False,0,...,0,0,1,0,0,1,0,0,0,1
1,0002_01,0,24.0,1.0,1.0,1.0,1.0,1.0,True,0,...,0,0,0,1,1,0,0,0,0,1
2,0003_01,0,58.0,1.0,1.0,0.0,1.0,1.0,False,1,...,0,0,0,1,0,1,0,0,0,1
3,0003_02,0,33.0,0.0,1.0,1.0,1.0,1.0,False,1,...,0,0,0,1,0,1,0,0,0,1
4,0004_01,0,16.0,1.0,1.0,1.0,1.0,1.0,True,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0,41.0,0.0,1.0,0.0,1.0,1.0,False,1,...,0,0,1,0,0,1,0,1,0,0
8689,9278_01,1,18.0,0.0,0.0,0.0,0.0,0.0,False,0,...,1,0,0,1,1,0,0,0,1,0
8690,9279_01,0,26.0,0.0,0.0,1.0,1.0,0.0,True,0,...,1,0,0,1,1,0,0,0,0,1
8691,9280_01,0,32.0,0.0,1.0,0.0,1.0,1.0,False,0,...,0,0,0,1,0,1,0,1,0,0


In [60]:
train_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8693 entries, 0 to 8692
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                8693 non-null   object 
 1   CryoSleep                  8693 non-null   int32  
 2   Age                        8693 non-null   float64
 3   RoomService                8693 non-null   float64
 4   FoodCourt                  8693 non-null   float64
 5   ShoppingMall               8693 non-null   float64
 6   Spa                        8693 non-null   float64
 7   VRDeck                     8693 non-null   float64
 8   Transported                8693 non-null   bool   
 9   Cabin1_A                   8693 non-null   uint8  
 10  Cabin1_B                   8693 non-null   uint8  
 11  Cabin1_C                   8693 non-null   uint8  
 12  Cabin1_D                   8693 non-null   uint8  
 13  Cabin1_E                   8693 non-null   uint8

# train_vip

In [61]:
vip

0       False
1       False
2        True
3       False
4       False
        ...  
8688     True
8689    False
8690    False
8691    False
8692    False
Name: VIP, Length: 8693, dtype: object

In [62]:
train_v = train_c

In [63]:
train_v

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0001_01,0,39.0,0.0,0.0,0.0,0.0,0.0,False,0,...,0,0,1,0,0,1,0,0,0,1
1,0002_01,0,24.0,1.0,1.0,1.0,1.0,1.0,True,0,...,0,0,0,1,1,0,0,0,0,1
2,0003_01,0,58.0,1.0,1.0,0.0,1.0,1.0,False,1,...,0,0,0,1,0,1,0,0,0,1
3,0003_02,0,33.0,0.0,1.0,1.0,1.0,1.0,False,1,...,0,0,0,1,0,1,0,0,0,1
4,0004_01,0,16.0,1.0,1.0,1.0,1.0,1.0,True,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0,41.0,0.0,1.0,0.0,1.0,1.0,False,1,...,0,0,1,0,0,1,0,1,0,0
8689,9278_01,1,18.0,0.0,0.0,0.0,0.0,0.0,False,0,...,1,0,0,1,1,0,0,0,1,0
8690,9279_01,0,26.0,0.0,0.0,1.0,1.0,0.0,True,0,...,1,0,0,1,1,0,0,0,0,1
8691,9280_01,0,32.0,0.0,1.0,0.0,1.0,1.0,False,0,...,0,0,0,1,0,1,0,1,0,0


In [64]:
vip = pd.DataFrame(vip)

In [65]:
vip.columns = ["VIP"]

In [66]:
vip

,VIP
0,False
1,False
2,True
3,False
4,False
...,...
8688,True
8689,False
8690,False
8691,False


In [67]:
train_v["VIP"] = vip["VIP"]

In [68]:
train_v

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,...,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP
0,0001_01,0,39.0,0.0,0.0,0.0,0.0,0.0,False,0,...,0,1,0,0,1,0,0,0,1,False
1,0002_01,0,24.0,1.0,1.0,1.0,1.0,1.0,True,0,...,0,0,1,1,0,0,0,0,1,False
2,0003_01,0,58.0,1.0,1.0,0.0,1.0,1.0,False,1,...,0,0,1,0,1,0,0,0,1,True
3,0003_02,0,33.0,0.0,1.0,1.0,1.0,1.0,False,1,...,0,0,1,0,1,0,0,0,1,False
4,0004_01,0,16.0,1.0,1.0,1.0,1.0,1.0,True,0,...,0,0,1,1,0,0,0,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0,41.0,0.0,1.0,0.0,1.0,1.0,False,1,...,0,1,0,0,1,0,1,0,0,True
8689,9278_01,1,18.0,0.0,0.0,0.0,0.0,0.0,False,0,...,0,0,1,1,0,0,0,1,0,False
8690,9279_01,0,26.0,0.0,0.0,1.0,1.0,0.0,True,0,...,0,0,1,1,0,0,0,0,1,False
8691,9280_01,0,32.0,0.0,1.0,0.0,1.0,1.0,False,0,...,0,0,1,0,1,0,1,0,0,False


In [69]:
train_v.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8693 entries, 0 to 8692
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                8693 non-null   object 
 1   CryoSleep                  8693 non-null   int32  
 2   Age                        8693 non-null   float64
 3   RoomService                8693 non-null   float64
 4   FoodCourt                  8693 non-null   float64
 5   ShoppingMall               8693 non-null   float64
 6   Spa                        8693 non-null   float64
 7   VRDeck                     8693 non-null   float64
 8   Transported                8693 non-null   bool   
 9   Cabin1_A                   8693 non-null   uint8  
 10  Cabin1_B                   8693 non-null   uint8  
 11  Cabin1_C                   8693 non-null   uint8  
 12  Cabin1_D                   8693 non-null   uint8  
 13  Cabin1_E                   8693 non-null   uint8

In [70]:
train_v_test = train_v[train_v.isnull().T.any()]

In [71]:
train_v_train = train_v.dropna()

In [72]:
train_v_test

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,...,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP
38,0036_01,0,15.00000,0.0,1.0,1.0,1.0,1.0,False,0,...,0,0,1,1,0,0,1,0,0,NaN
102,0108_03,0,0.00000,0.0,0.0,0.0,0.0,0.0,True,0,...,0,0,1,1,0,0,0,0,1,NaN
145,0165_01,1,35.00000,0.0,0.0,0.0,0.0,0.0,True,0,...,0,1,0,0,0,1,0,0,1,NaN
228,0244_02,1,14.00000,0.0,0.0,0.0,0.0,0.0,True,0,...,0,0,1,0,0,1,1,0,0,NaN
566,0593_01,0,28.82793,1.0,1.0,1.0,1.0,1.0,False,0,...,0,1,0,0,0,1,0,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8494,9074_01,1,0.00000,0.0,0.0,0.0,0.0,0.0,True,0,...,0,0,1,1,0,0,0,0,1,NaN
8512,9081_06,0,16.00000,0.0,0.0,1.0,0.0,0.0,False,0,...,0,1,0,1,0,0,0,1,0,NaN
8542,9122_01,1,55.00000,0.0,0.0,0.0,0.0,0.0,False,0,...,0,0,1,1,0,0,1,0,0,NaN
8630,9205_03,1,52.00000,0.0,0.0,0.0,0.0,0.0,True,0,...,0,1,0,0,1,0,0,0,1,NaN


In [73]:
train_v_train["VIP"] = train_v_train["VIP"].astype('int')

<ipython-input-73-245a08e46fde>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_v_train["VIP"] = train_v_train["VIP"].astype('int')


In [74]:
train_v_train

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,...,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP
0,0001_01,0,39.0,0.0,0.0,0.0,0.0,0.0,False,0,...,0,1,0,0,1,0,0,0,1,0
1,0002_01,0,24.0,1.0,1.0,1.0,1.0,1.0,True,0,...,0,0,1,1,0,0,0,0,1,0
2,0003_01,0,58.0,1.0,1.0,0.0,1.0,1.0,False,1,...,0,0,1,0,1,0,0,0,1,1
3,0003_02,0,33.0,0.0,1.0,1.0,1.0,1.0,False,1,...,0,0,1,0,1,0,0,0,1,0
4,0004_01,0,16.0,1.0,1.0,1.0,1.0,1.0,True,0,...,0,0,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0,41.0,0.0,1.0,0.0,1.0,1.0,False,1,...,0,1,0,0,1,0,1,0,0,1
8689,9278_01,1,18.0,0.0,0.0,0.0,0.0,0.0,False,0,...,0,0,1,1,0,0,0,1,0,0
8690,9279_01,0,26.0,0.0,0.0,1.0,1.0,0.0,True,0,...,0,0,1,1,0,0,0,0,1,0
8691,9280_01,0,32.0,0.0,1.0,0.0,1.0,1.0,False,0,...,0,0,1,0,1,0,1,0,0,0


In [75]:
train_v_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 203 entries, 38 to 8666
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                203 non-null    object 
 1   CryoSleep                  203 non-null    int32  
 2   Age                        203 non-null    float64
 3   RoomService                203 non-null    float64
 4   FoodCourt                  203 non-null    float64
 5   ShoppingMall               203 non-null    float64
 6   Spa                        203 non-null    float64
 7   VRDeck                     203 non-null    float64
 8   Transported                203 non-null    bool   
 9   Cabin1_A                   203 non-null    uint8  
 10  Cabin1_B                   203 non-null    uint8  
 11  Cabin1_C                   203 non-null    uint8  
 12  Cabin1_D                   203 non-null    uint8  
 13  Cabin1_E                   203 non-null    uint8

In [76]:
y_train_2 = train_v_train["VIP"]

In [77]:
y_train_2

0       0
1       0
2       1
3       0
4       0
       ..
8688    1
8689    0
8690    0
8691    0
8692    0
Name: VIP, Length: 8490, dtype: int32

In [78]:
df_train_v = train_v_train.drop(["VIP","PassengerId"], axis = 1)
df_train_v.astype('int')

,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,Cabin1_B,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0,39,0,0,0,0,0,0,0,1,...,0,0,1,0,0,1,0,0,0,1
1,0,24,1,1,1,1,1,1,0,0,...,0,0,0,1,1,0,0,0,0,1
2,0,58,1,1,0,1,1,0,1,0,...,0,0,0,1,0,1,0,0,0,1
3,0,33,0,1,1,1,1,0,1,0,...,0,0,0,1,0,1,0,0,0,1
4,0,16,1,1,1,1,1,1,0,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0,41,0,1,0,1,1,0,1,0,...,0,0,1,0,0,1,0,1,0,0
8689,1,18,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,1,0
8690,0,26,0,0,1,1,0,1,0,0,...,1,0,0,1,1,0,0,0,0,1
8691,0,32,0,1,0,1,1,0,0,0,...,0,0,0,1,0,1,0,1,0,0


In [79]:
X_train_2 = df_train_v

In [80]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state = 0) 
tree.fit(X_train_2, y_train_2) 
print("Accuracy on training set: {:.3f}".format(tree.score(X_train_2, y_train_2))) 

Accuracy on training set: 0.996


In [81]:
df_test_v = train_v_test.drop(["VIP","PassengerId"], axis = 1)
df_test_v.astype('int')

,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,Cabin1_B,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
38,0,15,0,1,1,1,1,0,0,0,...,0,0,0,1,1,0,0,1,0,0
102,0,0,0,0,0,0,0,1,0,0,...,1,0,0,1,1,0,0,0,0,1
145,1,35,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,1
228,1,14,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,1,0,0
566,0,28,1,1,1,1,1,0,0,0,...,0,0,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8494,1,0,0,0,0,0,0,1,0,0,...,1,0,0,1,1,0,0,0,0,1
8512,0,16,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0
8542,1,55,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,1,0,0
8630,1,52,0,0,0,0,0,1,0,1,...,0,0,1,0,0,1,0,0,0,1


In [82]:
X_test_2 = df_test_v

In [83]:
y_pred_2 = tree.predict(X_test_2) 
print("Test set predictions:\n {}".format(y_pred_2))

Test set predictions:
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [84]:
y_pred_2 = pd.DataFrame(y_pred_2)

In [85]:
y_pred_2.columns = ["VIP"]
y_pred_2

,VIP
0,0
1,0
2,0
3,0
4,0
...,...
198,0
199,0
200,0
201,0


In [86]:
y_pred_3 = y_pred_2.set_index(train_v_test.index)

In [87]:
y_pred_3

,VIP
38,0
102,0
145,0
228,0
566,0
...,...
8494,0
8512,0
8542,0
8630,0


In [88]:
train_v_test = train_v_test.fillna(y_pred_3)

In [89]:
train_v_test["VIP"] = train_v_test["VIP"].astype("int")

In [90]:
train_v_test 

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,...,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP
38,0036_01,0,15.00000,0.0,1.0,1.0,1.0,1.0,False,0,...,0,0,1,1,0,0,1,0,0,0
102,0108_03,0,0.00000,0.0,0.0,0.0,0.0,0.0,True,0,...,0,0,1,1,0,0,0,0,1,0
145,0165_01,1,35.00000,0.0,0.0,0.0,0.0,0.0,True,0,...,0,1,0,0,0,1,0,0,1,0
228,0244_02,1,14.00000,0.0,0.0,0.0,0.0,0.0,True,0,...,0,0,1,0,0,1,1,0,0,0
566,0593_01,0,28.82793,1.0,1.0,1.0,1.0,1.0,False,0,...,0,1,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8494,9074_01,1,0.00000,0.0,0.0,0.0,0.0,0.0,True,0,...,0,0,1,1,0,0,0,0,1,0
8512,9081_06,0,16.00000,0.0,0.0,1.0,0.0,0.0,False,0,...,0,1,0,1,0,0,0,1,0,0
8542,9122_01,1,55.00000,0.0,0.0,0.0,0.0,0.0,False,0,...,0,0,1,1,0,0,1,0,0,0
8630,9205_03,1,52.00000,0.0,0.0,0.0,0.0,0.0,True,0,...,0,1,0,0,1,0,0,0,1,0


In [91]:
train_v_train

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,...,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP
0,0001_01,0,39.0,0.0,0.0,0.0,0.0,0.0,False,0,...,0,1,0,0,1,0,0,0,1,0
1,0002_01,0,24.0,1.0,1.0,1.0,1.0,1.0,True,0,...,0,0,1,1,0,0,0,0,1,0
2,0003_01,0,58.0,1.0,1.0,0.0,1.0,1.0,False,1,...,0,0,1,0,1,0,0,0,1,1
3,0003_02,0,33.0,0.0,1.0,1.0,1.0,1.0,False,1,...,0,0,1,0,1,0,0,0,1,0
4,0004_01,0,16.0,1.0,1.0,1.0,1.0,1.0,True,0,...,0,0,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0,41.0,0.0,1.0,0.0,1.0,1.0,False,1,...,0,1,0,0,1,0,1,0,0,1
8689,9278_01,1,18.0,0.0,0.0,0.0,0.0,0.0,False,0,...,0,0,1,1,0,0,0,1,0,0
8690,9279_01,0,26.0,0.0,0.0,1.0,1.0,0.0,True,0,...,0,0,1,1,0,0,0,0,1,0
8691,9280_01,0,32.0,0.0,1.0,0.0,1.0,1.0,False,0,...,0,0,1,0,1,0,1,0,0,0


In [92]:
train_v = pd.concat([train_v_train, train_v_test],axis = 0)

In [93]:
train_v = train_v.sort_index()

In [94]:
train_v

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Cabin1_A,...,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP
0,0001_01,0,39.0,0.0,0.0,0.0,0.0,0.0,False,0,...,0,1,0,0,1,0,0,0,1,0
1,0002_01,0,24.0,1.0,1.0,1.0,1.0,1.0,True,0,...,0,0,1,1,0,0,0,0,1,0
2,0003_01,0,58.0,1.0,1.0,0.0,1.0,1.0,False,1,...,0,0,1,0,1,0,0,0,1,1
3,0003_02,0,33.0,0.0,1.0,1.0,1.0,1.0,False,1,...,0,0,1,0,1,0,0,0,1,0
4,0004_01,0,16.0,1.0,1.0,1.0,1.0,1.0,True,0,...,0,0,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0,41.0,0.0,1.0,0.0,1.0,1.0,False,1,...,0,1,0,0,1,0,1,0,0,1
8689,9278_01,1,18.0,0.0,0.0,0.0,0.0,0.0,False,0,...,0,0,1,1,0,0,0,1,0,0
8690,9279_01,0,26.0,0.0,0.0,1.0,1.0,0.0,True,0,...,0,0,1,1,0,0,0,0,1,0
8691,9280_01,0,32.0,0.0,1.0,0.0,1.0,1.0,False,0,...,0,0,1,0,1,0,1,0,0,0


In [95]:
train_v.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8693 entries, 0 to 8692
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                8693 non-null   object 
 1   CryoSleep                  8693 non-null   int32  
 2   Age                        8693 non-null   float64
 3   RoomService                8693 non-null   float64
 4   FoodCourt                  8693 non-null   float64
 5   ShoppingMall               8693 non-null   float64
 6   Spa                        8693 non-null   float64
 7   VRDeck                     8693 non-null   float64
 8   Transported                8693 non-null   bool   
 9   Cabin1_A                   8693 non-null   uint8  
 10  Cabin1_B                   8693 non-null   uint8  
 11  Cabin1_C                   8693 non-null   uint8  
 12  Cabin1_D                   8693 non-null   uint8  
 13  Cabin1_E                   8693 non-null   uint8

In [96]:
test

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0013_01,True,27.000000,False,0.0,0.0,0.0,0.0,0.0,0,...,0,0,1,0,0,1,0,0,0,1
1,0018_01,False,19.000000,False,0.0,1.0,0.0,1.0,0.0,0,...,0,0,0,1,1,0,0,0,0,1
2,0019_01,True,31.000000,False,0.0,0.0,0.0,0.0,0.0,1,...,0,0,0,1,0,1,0,0,0,1
3,0021_01,False,38.000000,False,0.0,1.0,0.0,1.0,1.0,1,...,0,0,0,1,0,1,0,0,0,1
4,0023_01,False,20.000000,False,1.0,0.0,1.0,0.0,0.0,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,True,34.000000,False,0.0,0.0,0.0,0.0,0.0,0,...,0,0,0,1,1,0,0,0,0,1
4273,9269_01,False,42.000000,False,0.0,1.0,1.0,1.0,1.0,0,...,0,0,0,1,1,0,0,0,0,1
4274,9271_01,True,28.658146,False,0.0,0.0,0.0,0.0,0.0,0,...,0,0,1,0,0,0,1,0,0,1
4275,9273_01,False,28.658146,False,0.0,1.0,0.0,0.0,1.0,0,...,0,0,1,0,0,1,0,0,1,0


In [97]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                4277 non-null   object 
 1   CryoSleep                  4184 non-null   object 
 2   Age                        4277 non-null   float64
 3   VIP                        4184 non-null   object 
 4   RoomService                4277 non-null   float64
 5   FoodCourt                  4277 non-null   float64
 6   ShoppingMall               4277 non-null   float64
 7   Spa                        4277 non-null   float64
 8   VRDeck                     4277 non-null   float64
 9   Cabin1_A                   4277 non-null   uint8  
 10  Cabin1_B                   4277 non-null   uint8  
 11  Cabin1_C                   4277 non-null   uint8  
 12  Cabin1_D                   4277 non-null   uint8  
 13  Cabin1_E                   4277 non-null   uint8

# test_c

In [98]:
test_c = test
test_c.drop("VIP",inplace = True, axis = 1)

In [99]:
test_c.head()

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0013_01,True,27.0,0.0,0.0,0.0,0.0,0.0,0,1,...,0,0,1,0,0,1,0,0,0,1
1,0018_01,False,19.0,0.0,1.0,0.0,1.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,1
2,0019_01,True,31.0,0.0,0.0,0.0,0.0,0.0,1,0,...,0,0,0,1,0,1,0,0,0,1
3,0021_01,False,38.0,0.0,1.0,0.0,1.0,1.0,1,0,...,0,0,0,1,0,1,0,0,0,1
4,0023_01,False,20.0,1.0,0.0,1.0,0.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,1


In [100]:
test_c_test = test_c[test_c.isnull().T.any()]

In [101]:
test_c_train = test_c.dropna()

In [102]:
test_c_train

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0013_01,True,27.000000,0.0,0.0,0.0,0.0,0.0,0,1,...,0,0,1,0,0,1,0,0,0,1
1,0018_01,False,19.000000,0.0,1.0,0.0,1.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,1
2,0019_01,True,31.000000,0.0,0.0,0.0,0.0,0.0,1,0,...,0,0,0,1,0,1,0,0,0,1
3,0021_01,False,38.000000,0.0,1.0,0.0,1.0,1.0,1,0,...,0,0,0,1,0,1,0,0,0,1
4,0023_01,False,20.000000,1.0,0.0,1.0,0.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,True,34.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,1
4273,9269_01,False,42.000000,0.0,1.0,1.0,1.0,1.0,0,0,...,0,0,0,1,1,0,0,0,0,1
4274,9271_01,True,28.658146,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,1,0,0,0,1,0,0,1
4275,9273_01,False,28.658146,0.0,1.0,0.0,0.0,1.0,0,0,...,0,0,1,0,0,1,0,0,1,0


In [103]:
test_c_test

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
68,0156_01,NaN,28.658146,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,1
78,0175_02,NaN,8.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,1,0,0,1,1,0,0,0,0,1
92,0208_01,NaN,40.000000,1.0,0.0,1.0,0.0,1.0,0,0,...,1,0,1,0,1,0,0,0,0,1
99,0227_01,NaN,22.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,0,0,1,0,0,1
165,0349_01,NaN,33.000000,0.0,1.0,0.0,1.0,1.0,0,0,...,1,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,8705_01,NaN,20.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,1,0,1,0,0,1,0,0
4108,8925_06,NaN,27.000000,0.0,1.0,1.0,1.0,1.0,0,0,...,0,0,1,0,0,0,1,0,1,0
4209,9138_01,NaN,41.000000,0.0,1.0,0.0,1.0,1.0,0,0,...,0,0,1,0,0,0,1,0,0,1
4230,9182_01,NaN,14.000000,0.0,0.0,0.0,1.0,1.0,0,0,...,1,0,0,1,0,0,0,0,0,1


In [104]:
test_c_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93 entries, 68 to 4239
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                93 non-null     object 
 1   CryoSleep                  0 non-null      object 
 2   Age                        93 non-null     float64
 3   RoomService                93 non-null     float64
 4   FoodCourt                  93 non-null     float64
 5   ShoppingMall               93 non-null     float64
 6   Spa                        93 non-null     float64
 7   VRDeck                     93 non-null     float64
 8   Cabin1_A                   93 non-null     uint8  
 9   Cabin1_B                   93 non-null     uint8  
 10  Cabin1_C                   93 non-null     uint8  
 11  Cabin1_D                   93 non-null     uint8  
 12  Cabin1_E                   93 non-null     uint8  
 13  Cabin1_F                   93 non-null     uint8 

In [105]:
y_train_4 = test_c_train["CryoSleep"].astype('int')

In [106]:
y_train_4

0       1
1       0
2       1
3       0
4       0
       ..
4272    1
4273    0
4274    1
4275    0
4276    1
Name: CryoSleep, Length: 4184, dtype: int32

In [107]:
df_test_c_train = test_c_train.drop(["CryoSleep","PassengerId"], axis = 1)
df_test_c_train.astype('int')

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,Cabin1_C,Cabin1_D,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,27,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,1
1,19,0,1,0,1,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
2,31,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,1
3,38,0,1,0,1,1,1,0,0,0,...,0,0,0,1,0,1,0,0,0,1
4,20,1,0,1,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,34,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
4273,42,0,1,1,1,1,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
4274,28,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
4275,28,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0


In [108]:
X_train_4 = df_test_c_train

In [109]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state = 0) 
tree.fit(X_train_4, y_train_4) 
print("Accuracy on training set: {:.3f}".format(tree.score(X_train_4, y_train_4))) 

Accuracy on training set: 0.979


In [110]:
df_test_c_test = test_c_test.drop(["CryoSleep","PassengerId"], axis = 1)
df_test_c_test.astype('int')

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,Cabin1_C,Cabin1_D,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
68,28,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
78,8,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,0,1
92,40,1,0,1,0,1,0,0,0,0,...,1,0,1,0,1,0,0,0,0,1
99,22,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,1,0,0,1
165,33,0,1,0,1,1,0,0,0,0,...,1,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,20,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,1,0,0
4108,27,0,1,1,1,1,0,0,0,0,...,0,0,1,0,0,0,1,0,1,0
4209,41,0,1,0,1,1,0,0,0,1,...,0,0,1,0,0,0,1,0,0,1
4230,14,0,0,0,1,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,1


In [111]:
X_test_4 = df_test_c_test

In [112]:
y_pred_4 = tree.predict(X_test_4) 
print("Test set predictions:\n {}".format(y_pred_4))

Test set predictions:
 [1 0 0 1 0 1 0 1 1 0 1 1 0 0 1 0 0 1 0 0 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 1
 0 0 1 1 0 1 1 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 1 0 0 1
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1]


In [113]:
y_pred_4 = pd.DataFrame(y_pred_4)

In [114]:
y_pred_4.columns = ["CryoSleep"]
y_pred_4

,CryoSleep
0,1
1,0
2,0
3,1
4,0
...,...
88,1
89,0
90,0
91,0


In [115]:
y_pred_5 = y_pred_4.set_index(test_c_test.index)

In [116]:
y_pred_5

,CryoSleep
68,1
78,0
92,0
99,1
165,0
...,...
3995,1
4108,0
4209,0
4230,0


In [117]:
test_c_test = test_c_test.fillna(y_pred_5)

In [118]:
test_c_test

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
68,0156_01,1.0,28.658146,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,1
78,0175_02,0.0,8.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,1,0,0,1,1,0,0,0,0,1
92,0208_01,0.0,40.000000,1.0,0.0,1.0,0.0,1.0,0,0,...,1,0,1,0,1,0,0,0,0,1
99,0227_01,1.0,22.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,0,0,1,0,0,1
165,0349_01,0.0,33.000000,0.0,1.0,0.0,1.0,1.0,0,0,...,1,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,8705_01,1.0,20.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,1,0,1,0,0,1,0,0
4108,8925_06,0.0,27.000000,0.0,1.0,1.0,1.0,1.0,0,0,...,0,0,1,0,0,0,1,0,1,0
4209,9138_01,0.0,41.000000,0.0,1.0,0.0,1.0,1.0,0,0,...,0,0,1,0,0,0,1,0,0,1
4230,9182_01,0.0,14.000000,0.0,0.0,0.0,1.0,1.0,0,0,...,1,0,0,1,0,0,0,0,0,1


In [119]:
test_c_test["CryoSleep"] = test_c_test["CryoSleep"].astype("int")

In [120]:
test_c_test

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
68,0156_01,1,28.658146,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,1
78,0175_02,0,8.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,1,0,0,1,1,0,0,0,0,1
92,0208_01,0,40.000000,1.0,0.0,1.0,0.0,1.0,0,0,...,1,0,1,0,1,0,0,0,0,1
99,0227_01,1,22.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,0,0,1,0,0,1
165,0349_01,0,33.000000,0.0,1.0,0.0,1.0,1.0,0,0,...,1,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,8705_01,1,20.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,1,0,1,0,0,1,0,0
4108,8925_06,0,27.000000,0.0,1.0,1.0,1.0,1.0,0,0,...,0,0,1,0,0,0,1,0,1,0
4209,9138_01,0,41.000000,0.0,1.0,0.0,1.0,1.0,0,0,...,0,0,1,0,0,0,1,0,0,1
4230,9182_01,0,14.000000,0.0,0.0,0.0,1.0,1.0,0,0,...,1,0,0,1,0,0,0,0,0,1


In [121]:
test_c_train["CryoSleep"] = test_c_train["CryoSleep"].astype("int")

<ipython-input-121-fc45fd02aec7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_c_train["CryoSleep"] = test_c_train["CryoSleep"].astype("int")


In [122]:
test_c_train

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0013_01,1,27.000000,0.0,0.0,0.0,0.0,0.0,0,1,...,0,0,1,0,0,1,0,0,0,1
1,0018_01,0,19.000000,0.0,1.0,0.0,1.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,1
2,0019_01,1,31.000000,0.0,0.0,0.0,0.0,0.0,1,0,...,0,0,0,1,0,1,0,0,0,1
3,0021_01,0,38.000000,0.0,1.0,0.0,1.0,1.0,1,0,...,0,0,0,1,0,1,0,0,0,1
4,0023_01,0,20.000000,1.0,0.0,1.0,0.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,1,34.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,1
4273,9269_01,0,42.000000,0.0,1.0,1.0,1.0,1.0,0,0,...,0,0,0,1,1,0,0,0,0,1
4274,9271_01,1,28.658146,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,1,0,0,0,1,0,0,1
4275,9273_01,0,28.658146,0.0,1.0,0.0,0.0,1.0,0,0,...,0,0,1,0,0,1,0,0,1,0


In [123]:
test_c = pd.concat([test_c_train, test_c_test],axis = 0)

In [124]:
test_c = test_c.sort_index()

In [125]:
test_c

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0013_01,1,27.000000,0.0,0.0,0.0,0.0,0.0,0,1,...,0,0,1,0,0,1,0,0,0,1
1,0018_01,0,19.000000,0.0,1.0,0.0,1.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,1
2,0019_01,1,31.000000,0.0,0.0,0.0,0.0,0.0,1,0,...,0,0,0,1,0,1,0,0,0,1
3,0021_01,0,38.000000,0.0,1.0,0.0,1.0,1.0,1,0,...,0,0,0,1,0,1,0,0,0,1
4,0023_01,0,20.000000,1.0,0.0,1.0,0.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,1,34.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,1
4273,9269_01,0,42.000000,0.0,1.0,1.0,1.0,1.0,0,0,...,0,0,0,1,1,0,0,0,0,1
4274,9271_01,1,28.658146,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,1,0,0,0,1,0,0,1
4275,9273_01,0,28.658146,0.0,1.0,0.0,0.0,1.0,0,0,...,0,0,1,0,0,1,0,0,1,0


In [126]:
test_c.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4277 entries, 0 to 4276
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                4277 non-null   object 
 1   CryoSleep                  4277 non-null   int32  
 2   Age                        4277 non-null   float64
 3   RoomService                4277 non-null   float64
 4   FoodCourt                  4277 non-null   float64
 5   ShoppingMall               4277 non-null   float64
 6   Spa                        4277 non-null   float64
 7   VRDeck                     4277 non-null   float64
 8   Cabin1_A                   4277 non-null   uint8  
 9   Cabin1_B                   4277 non-null   uint8  
 10  Cabin1_C                   4277 non-null   uint8  
 11  Cabin1_D                   4277 non-null   uint8  
 12  Cabin1_E                   4277 non-null   uint8  
 13  Cabin1_F                   4277 non-null   uint8

# test_vip

In [127]:
vip2

0       False
1       False
2       False
3       False
4       False
        ...  
4272    False
4273    False
4274    False
4275    False
4276    False
Name: VIP, Length: 4277, dtype: object

In [128]:
test_v = test_c

In [129]:
test_v

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0013_01,1,27.000000,0.0,0.0,0.0,0.0,0.0,0,1,...,0,0,1,0,0,1,0,0,0,1
1,0018_01,0,19.000000,0.0,1.0,0.0,1.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,1
2,0019_01,1,31.000000,0.0,0.0,0.0,0.0,0.0,1,0,...,0,0,0,1,0,1,0,0,0,1
3,0021_01,0,38.000000,0.0,1.0,0.0,1.0,1.0,1,0,...,0,0,0,1,0,1,0,0,0,1
4,0023_01,0,20.000000,1.0,0.0,1.0,0.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,1,34.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,1,0,0,0,0,1
4273,9269_01,0,42.000000,0.0,1.0,1.0,1.0,1.0,0,0,...,0,0,0,1,1,0,0,0,0,1
4274,9271_01,1,28.658146,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,1,0,0,0,1,0,0,1
4275,9273_01,0,28.658146,0.0,1.0,0.0,0.0,1.0,0,0,...,0,0,1,0,0,1,0,0,1,0


In [130]:
vip2 = pd.DataFrame(vip2)

In [131]:
vip2.columns = ["VIP"]

In [132]:
vip2

,VIP
0,False
1,False
2,False
3,False
4,False
...,...
4272,False
4273,False
4274,False
4275,False


In [133]:
test_v["VIP"] = vip2["VIP"]

In [134]:
test_v

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,...,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP
0,0013_01,1,27.000000,0.0,0.0,0.0,0.0,0.0,0,1,...,0,1,0,0,1,0,0,0,1,False
1,0018_01,0,19.000000,0.0,1.0,0.0,1.0,0.0,0,0,...,0,0,1,1,0,0,0,0,1,False
2,0019_01,1,31.000000,0.0,0.0,0.0,0.0,0.0,1,0,...,0,0,1,0,1,0,0,0,1,False
3,0021_01,0,38.000000,0.0,1.0,0.0,1.0,1.0,1,0,...,0,0,1,0,1,0,0,0,1,False
4,0023_01,0,20.000000,1.0,0.0,1.0,0.0,0.0,0,0,...,0,0,1,1,0,0,0,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,1,34.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,1,1,0,0,0,0,1,False
4273,9269_01,0,42.000000,0.0,1.0,1.0,1.0,1.0,0,0,...,0,0,1,1,0,0,0,0,1,False
4274,9271_01,1,28.658146,0.0,0.0,0.0,0.0,0.0,0,0,...,0,1,0,0,0,1,0,0,1,False
4275,9273_01,0,28.658146,0.0,1.0,0.0,0.0,1.0,0,0,...,0,1,0,0,1,0,0,1,0,False


In [135]:
test_v.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4277 entries, 0 to 4276
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                4277 non-null   object 
 1   CryoSleep                  4277 non-null   int32  
 2   Age                        4277 non-null   float64
 3   RoomService                4277 non-null   float64
 4   FoodCourt                  4277 non-null   float64
 5   ShoppingMall               4277 non-null   float64
 6   Spa                        4277 non-null   float64
 7   VRDeck                     4277 non-null   float64
 8   Cabin1_A                   4277 non-null   uint8  
 9   Cabin1_B                   4277 non-null   uint8  
 10  Cabin1_C                   4277 non-null   uint8  
 11  Cabin1_D                   4277 non-null   uint8  
 12  Cabin1_E                   4277 non-null   uint8  
 13  Cabin1_F                   4277 non-null   uint8

In [136]:
test_v_test = test_v[test_v.isnull().T.any()]

In [137]:
test_v_train = test_v.dropna()

In [138]:
test_v_test

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,...,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP
42,0100_01,0,28.658146,0.0,1.0,1.0,0.0,0.0,0,1,...,0,1,0,0,1,0,0,0,1,NaN
93,0209_01,0,36.000000,1.0,0.0,0.0,1.0,1.0,0,0,...,0,0,0,0,0,1,0,0,1,NaN
101,0228_02,0,39.000000,0.0,1.0,0.0,1.0,1.0,0,0,...,0,0,1,1,0,0,0,0,1,NaN
148,0325_01,0,48.000000,0.0,1.0,0.0,0.0,0.0,0,0,...,0,1,0,1,0,0,0,0,1,NaN
150,0329_01,1,33.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,1,1,0,0,0,1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4077,8875_02,0,32.000000,1.0,1.0,0.0,1.0,1.0,0,0,...,0,0,1,0,1,0,0,0,1,NaN
4087,8895_01,1,39.000000,0.0,0.0,0.0,0.0,0.0,0,1,...,0,0,1,0,1,0,0,0,1,NaN
4176,9058_01,0,38.000000,1.0,0.0,1.0,1.0,1.0,0,0,...,0,1,0,1,0,0,0,0,1,NaN
4250,9228_01,0,21.000000,1.0,0.0,1.0,0.0,1.0,0,0,...,0,1,0,1,0,0,0,0,1,NaN


In [139]:
test_v_train["VIP"] = test_v_train["VIP"].astype('int')

<ipython-input-139-45d11f94f8d0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_v_train["VIP"] = test_v_train["VIP"].astype('int')


In [140]:
test_v_train

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,...,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP
0,0013_01,1,27.000000,0.0,0.0,0.0,0.0,0.0,0,1,...,0,1,0,0,1,0,0,0,1,0
1,0018_01,0,19.000000,0.0,1.0,0.0,1.0,0.0,0,0,...,0,0,1,1,0,0,0,0,1,0
2,0019_01,1,31.000000,0.0,0.0,0.0,0.0,0.0,1,0,...,0,0,1,0,1,0,0,0,1,0
3,0021_01,0,38.000000,0.0,1.0,0.0,1.0,1.0,1,0,...,0,0,1,0,1,0,0,0,1,0
4,0023_01,0,20.000000,1.0,0.0,1.0,0.0,0.0,0,0,...,0,0,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,1,34.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,1,1,0,0,0,0,1,0
4273,9269_01,0,42.000000,0.0,1.0,1.0,1.0,1.0,0,0,...,0,0,1,1,0,0,0,0,1,0
4274,9271_01,1,28.658146,0.0,0.0,0.0,0.0,0.0,0,0,...,0,1,0,0,0,1,0,0,1,0
4275,9273_01,0,28.658146,0.0,1.0,0.0,0.0,1.0,0,0,...,0,1,0,0,1,0,0,1,0,0


In [141]:
test_v_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93 entries, 42 to 4267
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                93 non-null     object 
 1   CryoSleep                  93 non-null     int32  
 2   Age                        93 non-null     float64
 3   RoomService                93 non-null     float64
 4   FoodCourt                  93 non-null     float64
 5   ShoppingMall               93 non-null     float64
 6   Spa                        93 non-null     float64
 7   VRDeck                     93 non-null     float64
 8   Cabin1_A                   93 non-null     uint8  
 9   Cabin1_B                   93 non-null     uint8  
 10  Cabin1_C                   93 non-null     uint8  
 11  Cabin1_D                   93 non-null     uint8  
 12  Cabin1_E                   93 non-null     uint8  
 13  Cabin1_F                   93 non-null     uint8 

In [142]:
y_train_5 = test_v_train["VIP"]

In [143]:
y_train_5

0       0
1       0
2       0
3       0
4       0
       ..
4272    0
4273    0
4274    0
4275    0
4276    0
Name: VIP, Length: 4184, dtype: int32

In [144]:
df_test_v_train = test_v_train.drop(["VIP","PassengerId"], axis = 1)
df_test_v_train.astype('int')

,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,Cabin1_C,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,1,27,0,0,0,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,1
1,0,19,0,1,0,1,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
2,1,31,0,0,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,1
3,0,38,0,1,0,1,1,1,0,0,...,0,0,0,1,0,1,0,0,0,1
4,0,20,1,0,1,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,1,34,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
4273,0,42,0,1,1,1,1,0,0,0,...,0,0,0,1,1,0,0,0,0,1
4274,1,28,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
4275,0,28,0,1,0,0,1,0,0,1,...,0,0,1,0,0,1,0,0,1,0


In [145]:
X_train_5 = df_test_v_train

In [146]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state = 0) 
tree.fit(X_train_5, y_train_5) 
print("Accuracy on training set: {:.3f}".format(tree.score(X_train_5, y_train_5))) 

Accuracy on training set: 0.997


In [147]:
df_test_v_test = test_v_test.drop(["VIP","PassengerId"], axis = 1)
df_test_v_test.astype('int')

,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,Cabin1_C,...,Cabin1_G,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
42,0,28,0,1,1,0,0,0,1,0,...,0,0,1,0,0,1,0,0,0,1
93,0,36,1,0,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
101,0,39,0,1,0,1,1,0,0,0,...,1,0,0,1,1,0,0,0,0,1
148,0,48,0,1,0,0,0,0,0,0,...,1,0,1,0,1,0,0,0,0,1
150,1,33,0,0,0,0,0,0,0,0,...,1,0,0,1,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4077,0,32,1,1,0,1,1,0,0,1,...,0,0,0,1,0,1,0,0,0,1
4087,1,39,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,1
4176,0,38,1,0,1,1,1,0,0,0,...,0,0,1,0,1,0,0,0,0,1
4250,0,21,1,0,1,0,1,0,0,0,...,1,0,1,0,1,0,0,0,0,1


In [148]:
X_test_5 = df_test_v_test

In [149]:
y_pred_6 = tree.predict(X_test_5) 
print("Test set predictions:\n {}".format(y_pred_6))

Test set predictions:
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0]


In [150]:
y_pred_6 = pd.DataFrame(y_pred_6)

In [151]:
y_pred_6.columns = ["VIP"]
y_pred_6

,VIP
0,0
1,0
2,0
3,0
4,0
...,...
88,0
89,0
90,0
91,0


In [152]:
y_pred_7 = y_pred_6.set_index(test_v_test.index)

In [153]:
y_pred_7

,VIP
42,0
93,0
101,0
148,0
150,0
...,...
4077,0
4087,0
4176,0
4250,0


In [154]:
test_v_test = test_v_test.fillna(y_pred_7)

In [155]:
test_v_test["VIP"] = test_v_test["VIP"].astype("int")

In [156]:
test_v_test

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,...,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP
42,0100_01,0,28.658146,0.0,1.0,1.0,0.0,0.0,0,1,...,0,1,0,0,1,0,0,0,1,0
93,0209_01,0,36.000000,1.0,0.0,0.0,1.0,1.0,0,0,...,0,0,0,0,0,1,0,0,1,0
101,0228_02,0,39.000000,0.0,1.0,0.0,1.0,1.0,0,0,...,0,0,1,1,0,0,0,0,1,0
148,0325_01,0,48.000000,0.0,1.0,0.0,0.0,0.0,0,0,...,0,1,0,1,0,0,0,0,1,0
150,0329_01,1,33.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,1,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4077,8875_02,0,32.000000,1.0,1.0,0.0,1.0,1.0,0,0,...,0,0,1,0,1,0,0,0,1,0
4087,8895_01,1,39.000000,0.0,0.0,0.0,0.0,0.0,0,1,...,0,0,1,0,1,0,0,0,1,0
4176,9058_01,0,38.000000,1.0,0.0,1.0,1.0,1.0,0,0,...,0,1,0,1,0,0,0,0,1,0
4250,9228_01,0,21.000000,1.0,0.0,1.0,0.0,1.0,0,0,...,0,1,0,1,0,0,0,0,1,0


In [157]:
test_v_train

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,...,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP
0,0013_01,1,27.000000,0.0,0.0,0.0,0.0,0.0,0,1,...,0,1,0,0,1,0,0,0,1,0
1,0018_01,0,19.000000,0.0,1.0,0.0,1.0,0.0,0,0,...,0,0,1,1,0,0,0,0,1,0
2,0019_01,1,31.000000,0.0,0.0,0.0,0.0,0.0,1,0,...,0,0,1,0,1,0,0,0,1,0
3,0021_01,0,38.000000,0.0,1.0,0.0,1.0,1.0,1,0,...,0,0,1,0,1,0,0,0,1,0
4,0023_01,0,20.000000,1.0,0.0,1.0,0.0,0.0,0,0,...,0,0,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,1,34.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,1,1,0,0,0,0,1,0
4273,9269_01,0,42.000000,0.0,1.0,1.0,1.0,1.0,0,0,...,0,0,1,1,0,0,0,0,1,0
4274,9271_01,1,28.658146,0.0,0.0,0.0,0.0,0.0,0,0,...,0,1,0,0,0,1,0,0,1,0
4275,9273_01,0,28.658146,0.0,1.0,0.0,0.0,1.0,0,0,...,0,1,0,0,1,0,0,1,0,0


In [158]:
test_v = pd.concat([test_v_train, test_v_test],axis = 0)

In [159]:
test_v = test_v.sort_index()

In [160]:
test_v

,PassengerId,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,...,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP
0,0013_01,1,27.000000,0.0,0.0,0.0,0.0,0.0,0,1,...,0,1,0,0,1,0,0,0,1,0
1,0018_01,0,19.000000,0.0,1.0,0.0,1.0,0.0,0,0,...,0,0,1,1,0,0,0,0,1,0
2,0019_01,1,31.000000,0.0,0.0,0.0,0.0,0.0,1,0,...,0,0,1,0,1,0,0,0,1,0
3,0021_01,0,38.000000,0.0,1.0,0.0,1.0,1.0,1,0,...,0,0,1,0,1,0,0,0,1,0
4,0023_01,0,20.000000,1.0,0.0,1.0,0.0,0.0,0,0,...,0,0,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,1,34.000000,0.0,0.0,0.0,0.0,0.0,0,0,...,0,0,1,1,0,0,0,0,1,0
4273,9269_01,0,42.000000,0.0,1.0,1.0,1.0,1.0,0,0,...,0,0,1,1,0,0,0,0,1,0
4274,9271_01,1,28.658146,0.0,0.0,0.0,0.0,0.0,0,0,...,0,1,0,0,0,1,0,0,1,0
4275,9273_01,0,28.658146,0.0,1.0,0.0,0.0,1.0,0,0,...,0,1,0,0,1,0,0,1,0,0


In [161]:
test_v.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4277 entries, 0 to 4276
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   PassengerId                4277 non-null   object 
 1   CryoSleep                  4277 non-null   int32  
 2   Age                        4277 non-null   float64
 3   RoomService                4277 non-null   float64
 4   FoodCourt                  4277 non-null   float64
 5   ShoppingMall               4277 non-null   float64
 6   Spa                        4277 non-null   float64
 7   VRDeck                     4277 non-null   float64
 8   Cabin1_A                   4277 non-null   uint8  
 9   Cabin1_B                   4277 non-null   uint8  
 10  Cabin1_C                   4277 non-null   uint8  
 11  Cabin1_D                   4277 non-null   uint8  
 12  Cabin1_E                   4277 non-null   uint8  
 13  Cabin1_F                   4277 non-null   uint8

# model

In [162]:
y_train_3 = train_v['Transported']

In [163]:
y_train_3

0       False
1        True
2       False
3       False
4        True
        ...  
8688    False
8689    False
8690     True
8691    False
8692     True
Name: Transported, Length: 8693, dtype: bool

In [164]:
df_model = train_v.drop(["Transported","PassengerId"], axis = 1)
df_model

,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,Cabin1_C,...,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP
0,0,39.0,0.0,0.0,0.0,0.0,0.0,0,1,0,...,0,1,0,0,1,0,0,0,1,0
1,0,24.0,1.0,1.0,1.0,1.0,1.0,0,0,0,...,0,0,1,1,0,0,0,0,1,0
2,0,58.0,1.0,1.0,0.0,1.0,1.0,1,0,0,...,0,0,1,0,1,0,0,0,1,1
3,0,33.0,0.0,1.0,1.0,1.0,1.0,1,0,0,...,0,0,1,0,1,0,0,0,1,0
4,0,16.0,1.0,1.0,1.0,1.0,1.0,0,0,0,...,0,0,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0,41.0,0.0,1.0,0.0,1.0,1.0,1,0,0,...,0,1,0,0,1,0,1,0,0,1
8689,1,18.0,0.0,0.0,0.0,0.0,0.0,0,0,0,...,0,0,1,1,0,0,0,1,0,0
8690,0,26.0,0.0,0.0,1.0,1.0,0.0,0,0,0,...,0,0,1,1,0,0,0,0,1,0
8691,0,32.0,0.0,1.0,0.0,1.0,1.0,0,0,0,...,0,0,1,0,1,0,1,0,0,0


In [165]:
df_model.astype('int')

,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,Cabin1_C,...,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP
0,0,39,0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,1,0
1,0,24,1,1,1,1,1,0,0,0,...,0,0,1,1,0,0,0,0,1,0
2,0,58,1,1,0,1,1,1,0,0,...,0,0,1,0,1,0,0,0,1,1
3,0,33,0,1,1,1,1,1,0,0,...,0,0,1,0,1,0,0,0,1,0
4,0,16,1,1,1,1,1,0,0,0,...,0,0,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,0,41,0,1,0,1,1,1,0,0,...,0,1,0,0,1,0,1,0,0,1
8689,1,18,0,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,1,0,0
8690,0,26,0,0,1,1,0,0,0,0,...,0,0,1,1,0,0,0,0,1,0
8691,0,32,0,1,0,1,1,0,0,0,...,0,0,1,0,1,0,1,0,0,0


In [166]:
X_train_3 = df_model

In [167]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth = 8, random_state = 0) 
tree.fit(X_train_3, y_train_3) 
print("Accuracy on training set: {:.3f}".format(tree.score(X_train_3, y_train_3)))

Accuracy on training set: 0.773


In [168]:
from sklearn.linear_model import LogisticRegression 
logreg = LogisticRegression().fit(X_train_3, y_train_3) 
print("Training set score: {:.3f}".format(logreg.score(X_train_3, y_train_3))) 

Training set score: 0.743


C:\Users\yueli\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [169]:
from sklearn.svm import SVC
svc = SVC() 
svc.fit(X_train_3, y_train_3) 
print("Accuracy on training set: {:.2f}".format(svc.score(X_train_3, y_train_3))) 

Accuracy on training set: 0.74


In [170]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [171]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train_3, y_train_3) 
print("Accuracy on training set: {:.2f}".format(xgb.score(X_train_3, y_train_3))) 

Accuracy on training set: 0.83


# Final Results

In [172]:
df_model_test = test_v.drop(["PassengerId"], axis = 1)

In [173]:
X_test_3 = df_model_test

In [174]:
X_test_3

,CryoSleep,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cabin1_A,Cabin1_B,Cabin1_C,...,Cabin1_T,Cabin3_P,Cabin3_S,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,VIP
0,1,27.000000,0.0,0.0,0.0,0.0,0.0,0,1,0,...,0,1,0,0,1,0,0,0,1,0
1,0,19.000000,0.0,1.0,0.0,1.0,0.0,0,0,0,...,0,0,1,1,0,0,0,0,1,0
2,1,31.000000,0.0,0.0,0.0,0.0,0.0,1,0,0,...,0,0,1,0,1,0,0,0,1,0
3,0,38.000000,0.0,1.0,0.0,1.0,1.0,1,0,0,...,0,0,1,0,1,0,0,0,1,0
4,0,20.000000,1.0,0.0,1.0,0.0,0.0,0,0,0,...,0,0,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,1,34.000000,0.0,0.0,0.0,0.0,0.0,0,0,0,...,0,0,1,1,0,0,0,0,1,0
4273,0,42.000000,0.0,1.0,1.0,1.0,1.0,0,0,0,...,0,0,1,1,0,0,0,0,1,0
4274,1,28.658146,0.0,0.0,0.0,0.0,0.0,0,0,0,...,0,1,0,0,0,1,0,0,1,0
4275,0,28.658146,0.0,1.0,0.0,0.0,1.0,0,0,1,...,0,1,0,0,1,0,0,1,0,0


In [175]:
y_pred_3 = tree.predict(X_test_3) 
print("Test set predictions:\n {}".format(y_pred_3))

Test set predictions:
 [ True False  True ...  True False  True]


In [176]:
print("Accuracy on testing set: {:.3f}".format(tree.score(X_test_3, y_pred_3)))

Accuracy on testing set: 1.000


In [177]:
y_pred_3 = xgb.predict(X_test_3) 
print("Test set predictions:\n {}".format(y_pred_3))

Test set predictions:
 [1 0 1 ... 1 1 1]


In [178]:
y_pred_3 = pd.DataFrame(y_pred_3)

In [179]:
y_pred_3

,0
0,1
1,0
2,1
3,0
4,0
...,...
4272,1
4273,0
4274,1
4275,1


In [180]:
y_pred_3["PassengerId"] = test_v["PassengerId"]

In [183]:
y_pred_3["Transported"] = y_pred_3[0]

In [185]:
y_pred_3.drop(columns = 0, inplace = True)

In [186]:
y_pred_3

,PassengerId,Transported
0,0013_01,1
1,0018_01,0
2,0019_01,1
3,0021_01,0
4,0023_01,0
...,...,...
4272,9266_02,1
4273,9269_01,0
4274,9271_01,1
4275,9273_01,1


In [187]:
y_pred_3.to_csv("./spaceship.csv")